In [3]:
import collections
from datetime import date, timedelta
import os
from socket import timeout
import sys

import ee
from ee import ImageCollection
import geemap
import numpy as np
import pandas as pd
ee.Initialize()

# Test if shapefile works

In [4]:
!pwd

/home/hassib12/code/hassib12/FoodSecurityForecaster/notebooks


In [5]:
states_shp = f'../raw_data/USA/admin1/USA.shp'
ee_shape = geemap.shp_to_ee(states_shp)

/home/hassib12/.pyenv/versions/3.8.12/envs/FoodSecurityForecaster/lib/python3.8/site-packages/geemap/common.py:8493: UserWarning: The projection file /home/hassib12/code/hassib12/FoodSecurityForecaster/raw_data/USA/admin1/USA.prj could not be found. Assuming the dataset is in a geographic coordinate system (GCS).
  warnings.warn(


In [6]:
map = geemap.Map()
map.addLayer(ee_shape)

In [7]:
map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
state_names = pd.read_csv('../raw_data/state_names_USA.csv')['NAME1_,C,33']
state_names.head()

0       Alabama
1        Alaska
2       Arizona
3      Arkansas
4    California
Name: NAME1_,C,33, dtype: object

In [18]:
def download_map(country_code,
                 state_name,
                 date_range=('2021-01-01','2022-12-30'),
                 modis_collection='006/MOD13A1',
                 num_pixels=1000):
    Map = geemap.Map()
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/{country_code}/admin1/{lower_case_country_code}.shp'
 
    ee_shape = geemap.shp_to_ee(states_shp)
    state_ee = ee_shape.filter(ee.Filter.eq(["NAME1_"], state_name))
    Map.addLayer(state_ee.geometry())

    image_collection = ee.ImageCollection(f'MODIS/{modis_collection}').filter(ee.Filter.date(*date_range)).toBands()

    region = state_ee.geometry()

    samples = image_collection.sample(region, scale=300 , numPixels=num_pixels)

    features = samples.getInfo()['features']

    data = []
    for feature_set in features:
        for k, v in feature_set['properties'].items():
            date, property_name = k[:10], k[11:]
            date = date.replace('_', '-')
            data.append({'band': property_name, 'date': date, 'value': v})

    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df.attrs['state_name'] = state_name
    df['band'] = df['band'].astype("category")

    return df

def load_all(country_code='SSD', admin_level='admin1', date_range=('2010-01-01', '2018-01-01'), modis_collection='006/MOD13A1', num_pixels=1000):
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/USA/admin1/USA.shp'

    #ee_shape = geemap.shp_to_ee(states_shp)
    #geemap.common.ee_to_csv(ee_shape, "tmp.csv", timeout=1000)

    # state_names = pd.read_csv('tmp.csv')['NAME1_']
    # os.remove('tmp.csv') 
    
    for state_name in state_names:
        print(f'working on:{state_name}')
        df = download_map(country_code, state_name, date_range, modis_collection, num_pixels)
        df.to_pickle(f'../raw_data/raw_pixels/{state_name}.zip')
        print(f"Downloaded data for {state_name}...")

# if __name__ == "__main__":
#     ee.Initialize()
#     load_all(sys.argv[1])


In [19]:
ee_shape

In [20]:
load_all('USA')

working on:Alabama
The input shapefile could not be found.


AttributeError: 'NoneType' object has no attribute 'filter'

# SOUTH SUDAN

In [21]:
state_names = pd.read_csv('../raw_data/state_names_SSD.csv')['NAME_2,C,13']
state_names.head()

0    Bahr al Jabal
1        Kajo Kaii
2         Nahr Yei
3          Terkaka
4         Amatonge
Name: NAME_2,C,13, dtype: object

In [24]:
def download_map(country_code,
                 state_name,
                 date_range=('2021-01-01','2022-12-30'),
                 modis_collection='006/MOD13A1',
                 num_pixels=1000):
    Map = geemap.Map()
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/raw_data/gadm41_SSD_shp/gadm41_SSD_2.shp'
 
    ee_shape = geemap.shp_to_ee(states_shp)
    state_ee = ee_shape.filter(ee.Filter.eq(["NAME_2"], state_name))
    Map.addLayer(state_ee.geometry())

    image_collection = ee.ImageCollection(f'MODIS/{modis_collection}').filter(ee.Filter.date(*date_range)).toBands()

    region = state_ee.geometry()

    samples = image_collection.sample(region, scale=300 , numPixels=num_pixels)

    features = samples.getInfo()['features']

    data = []
    for feature_set in features:
        for k, v in feature_set['properties'].items():
            date, property_name = k[:10], k[11:]
            date = date.replace('_', '-')
            data.append({'band': property_name, 'date': date, 'value': v})

    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df.attrs['state_name'] = state_name
    df['band'] = df['band'].astype("category")

    return df

def load_all(country_code='SSD', admin_level='admin1', date_range=('2010-01-01', '2018-01-01'), modis_collection='006/MOD13A1', num_pixels=1000):
    lower_case_country_code = country_code.lower
    states_shp = f'../raw_data/raw_data/gadm41_SSD_shp/gadm41_SSD_2.shp'

    #ee_shape = geemap.shp_to_ee(states_shp)
    #geemap.common.ee_to_csv(ee_shape, "tmp.csv", timeout=1000)

    # state_names = pd.read_csv('tmp.csv')['NAME1_']
    # os.remove('tmp.csv') 
    
    for state_name in state_names:
        print(f'working on:{state_name}')
        df = download_map(country_code, state_name, date_range, modis_collection, num_pixels)
        df.to_pickle(f'../raw_data/raw_pixels/{state_name}.zip')
        print(f"Downloaded data for {state_name}...")

# if __name__ == "__main__":
#     ee.Initialize()
#     load_all(sys.argv[1])

In [25]:
load_all()

working on:Bahr al Jabal
The input shapefile could not be found.


AttributeError: 'NoneType' object has no attribute 'filter'